In [2]:
import torch
src = torch.arange(1, 11).reshape((2, 5))

src

tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10]])

In [7]:
index=torch.tensor([[0,1,2,0]])
index
torch.zeros(3,5,dtype=src.dtype).scatter_(0,index,src)

tensor([[1, 0, 0, 4, 0],
        [0, 2, 0, 0, 0],
        [0, 0, 3, 0, 0]])

In [8]:
index = torch.tensor([[0, 1, 2], [0, 1, 4]])
index

tensor([[0, 1, 2],
        [0, 1, 4]])

In [9]:
torch.zeros(3, 5, dtype=src.dtype).scatter_(1, index, src)

tensor([[1, 2, 3, 0, 0],
        [6, 7, 0, 0, 8],
        [0, 0, 0, 0, 0]])

In [10]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets,transforms


In [11]:
device=(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"using {device} device")

using cpu device


In [15]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten=nn.Flatten()
    self.linear_relu_stack=nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10),
    )

  def forward(self,x):
    x=self.flatten(x)
    logits=self.linear_relu_stack(x)
    return logits

In [17]:
model =NeuralNetwork().to(device)
print(model)
model.summary()

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


AttributeError: 'NeuralNetwork' object has no attribute 'summary'

In [19]:
model = NeuralNetwork().to(device)
# print(model)

# # Inspect the model architecture
# print("\nModel Architecture:")
# print(model)

# Count the number of trainable parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal Trainable Parameters: {total_params}")
#


Total Trainable Parameters: 669706


In [20]:

input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [21]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [24]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [25]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")


Before ReLU: tensor([[ 0.4857,  0.2944, -0.3220,  0.7770, -0.1137,  0.0069, -0.1268,  0.4720,
         -0.1371, -0.1976, -0.3170, -0.0294,  1.0712,  0.8125, -0.0120,  0.4465,
          0.1951, -0.3767,  0.3000, -0.0248],
        [ 0.5641,  0.3557, -0.2400,  0.4357,  0.5006, -0.1263, -0.3414,  0.6751,
          0.2707,  0.0240, -0.2792, -0.0185,  0.8168,  0.6627, -0.1758,  0.3921,
          0.0888, -0.5892,  0.2380, -0.0986],
        [ 0.8220,  0.4779, -0.3184,  0.2997,  0.0373, -0.7215, -0.3739,  0.5301,
          0.2174,  0.2377,  0.1048,  0.3826,  0.4962,  0.5219, -0.3899,  0.1065,
          0.4528, -0.6636,  0.1835,  0.0934]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.4857, 0.2944, 0.0000, 0.7770, 0.0000, 0.0069, 0.0000, 0.4720, 0.0000,
         0.0000, 0.0000, 0.0000, 1.0712, 0.8125, 0.0000, 0.4465, 0.1951, 0.0000,
         0.3000, 0.0000],
        [0.5641, 0.3557, 0.0000, 0.4357, 0.5006, 0.0000, 0.0000, 0.6751, 0.2707,
         0.0240, 0.0000, 0.0000, 0.8168, 0.6627, 0.00

In [26]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [29]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits
                      )
pred_probab

tensor([[0.1129, 0.1155, 0.0951, 0.1023, 0.0890, 0.0856, 0.1094, 0.0842, 0.0994,
         0.1065],
        [0.1345, 0.1003, 0.1052, 0.0909, 0.0781, 0.0811, 0.0877, 0.1061, 0.0898,
         0.1261],
        [0.1228, 0.0958, 0.0909, 0.1291, 0.0934, 0.0828, 0.1041, 0.0861, 0.0830,
         0.1121]], grad_fn=<SoftmaxBackward0>)

In [28]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0250, -0.0081,  0.0035,  ...,  0.0173, -0.0106,  0.0087],
        [ 0.0147, -0.0242, -0.0115,  ..., -0.0024,  0.0341, -0.0257]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0087, -0.0212], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0096, -0.0374,  0.0067,  ..., -0.0242,  0.0002, -0.0385],
        [ 0.0247,  0.0300,  0.0359,  ..., -0.0151,  0.0376,  0.0377]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 

In [35]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
w


tensor([[ 0.9152,  0.6287,  0.1940],
        [ 1.4140,  0.6090,  1.1242],
        [-0.5716, -0.9755,  0.4168],
        [ 0.8198,  0.1062, -0.3264],
        [ 0.5586,  0.4481, -0.4274]], requires_grad=True)

In [31]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x7f8507d3cf10>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7f8507d3e920>


In [36]:
loss.backward()
print(w.grad)
print(b.grad)
w

tensor([[0.3274, 0.2201, 0.3044],
        [0.3274, 0.2201, 0.3044],
        [0.3274, 0.2201, 0.3044],
        [0.3274, 0.2201, 0.3044],
        [0.3274, 0.2201, 0.3044]])
tensor([0.3274, 0.2201, 0.3044])


tensor([[ 0.9152,  0.6287,  0.1940],
        [ 1.4140,  0.6090,  1.1242],
        [-0.5716, -0.9755,  0.4168],
        [ 0.8198,  0.1062, -0.3264],
        [ 0.5586,  0.4481, -0.4274]], requires_grad=True)